In [1]:
## import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import csv
import skvideo.io
from tqdm import tqdm
%matplotlib inline

In [2]:
train = 'train'
test = 'test'

TRAIN_PATH = os.path.join('../',train)
TEST_PATH = os.path.join('../',test)

VID_PATH = 'videos/trash/'
IMG_PATH = 'photos/images/gray_Road_Only/'
TXT_PATH = 'text'
CSV_PATH = os.path.join(TXT_PATH, 'CSV')

TRAIN_VID = os.path.join(TRAIN_PATH,VID_PATH+'trainGray_Road_Only.m4v')
TEST_VID = os.path.join(TEST_PATH,VID_PATH+'testGray_Road_Only.m4v')

TRAIN_IMG = os.path.join(TRAIN_PATH,IMG_PATH)
TEST_IMG = os.path.join(TEST_PATH,IMG_PATH)

TRAIN_CSV = os.path.join(TRAIN_PATH,CSV_PATH)
TEST_CSV = os.path.join(TEST_PATH,CSV_PATH)

train_frames = 20400
test_frames = 10798

In [3]:
SPD_PATH = os.path.join(TRAIN_PATH, TXT_PATH)
train_y = list(pd.read_csv(os.path.join(SPD_PATH, 'train.txt'), header=None, squeeze=True))
assert(len(train_y)==train_frames)

In [4]:
def dataset_constructor(video_loc, img_folder, tot_frames, csv_loc, dataset_type):
    meta_dict = {}

    tqdm.write('reading in video file...')
    tot_frames = train_frames
    cap = skvideo.io.vread(video_loc)
     
    tqdm.write('constructing dataset...')
    for idx, frame in enumerate(tqdm(cap)):
        img_path = os.path.join(img_folder, str(idx)+'.jpg')
        frame_speed = float('NaN') if dataset_type == 'test' else train_y[idx]
        meta_dict[idx] = [img_path, idx, frame_speed]
        skvideo.io.vwrite(img_path, frame)
    meta_df = pd.DataFrame.from_dict(meta_dict, orient='index')
    meta_df.columns = ['image_path', 'image_index', 'speed']
    
    tqdm.write('writing meta to csv')
    meta_df.to_csv(os.path.join(csv_loc, dataset_type+'RoadOnly_meta.csv'), index=False)
    
    return "done dataset_constructor"

In [5]:
# train data
dataset_constructor(TRAIN_VID, TRAIN_IMG, train_frames, TRAIN_CSV, 'train')

reading in video file...


  0%|          | 2/20400 [00:00<30:34, 11.12it/s]

constructing dataset...


100%|██████████| 20400/20400 [31:54<00:00, 10.65it/s] 


writing meta to csv


'done dataset_constructor'

In [6]:
# test data
dataset_constructor(TEST_VID, TEST_IMG, test_frames, TEST_CSV, 'test')

reading in video file...


  0%|          | 2/10798 [00:00<12:04, 14.89it/s]

constructing dataset...


100%|██████████| 10798/10798 [13:27<00:00, 13.37it/s]


writing meta to csv


'done dataset_constructor'

In [ ]:
train_meta = pd.read_csv(os.path.join(TRAIN_CSV, 'train_meta.csv'))
assert(train_meta.shape[0] == train_frames)
assert(train_meta.shape[1] == 3)

In [ ]:
train_meta.head()

In [ ]:
for i in range(50):
    print('speed:',train_meta['speed'][i] )
    img=mpimg.imread(train_meta['image_path'][i])
    print('shape:', img.shape)
#     plt.imshow(img)
#     plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
plt.plot(train_meta['speed'])
plt.xlabel('image_index (or time since start)')
plt.ylabel('speed')
plt.title('Speed vs time')
plt.show()

In [ ]:
test_meta = pd.read_csv(os.path.join(TEST_CSV, 'test_meta.csv'))
assert(test_meta.shape[0] == test_frames)
assert(test_meta.shape[1] == 3)

In [ ]:
test_meta.head()

In [ ]:
for i in range(50):
    print('speed:',test_meta['speed'][i] )
    img=mpimg.imread(test_meta['image_path'][i])
    print('shape:', img.shape)
    plt.imshow(img)
    plt.show()